In [3]:
!nvidia-smi

Fri Jul 16 11:33:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.71       Driver Version: 456.71       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 166... WDDM  | 00000000:01:00.0  On |                  N/A |
| 28%   43C    P8    15W / 125W |   1542MiB /  6144MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

计算设备

In [4]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x27d9b9b4f98>)

查询可用gpu的数量

In [7]:
torch.cuda.device_count() # 这个torch

0

这两个函数允许我们在请求的GPU不存在的情况下运行代码

In [8]:
def try_gpu(i=0):  
    """如果存在，则返回gpu(i)，否则返回cpu()。"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]。"""
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cpu'), device(type='cpu'), [device(type='cpu')])

查询张量所在的设备

In [9]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

存储在GPU上

In [10]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]])

第二个GPU上创建一个随机张量

In [11]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.3827, 0.5812, 0.5293],
        [0.3628, 0.9810, 0.0076]])

要计算X + Y，我们需要决定在哪里执行这个操作

In [13]:
Z = X.cuda(1) # 先把X的值放在1号GPU上，赋值给Z
print(X)
print(Z)

AssertionError: Torch not compiled with CUDA enabled

现在数据在同一个GPU上（Z和Y都在），我们可以将它们相加

In [14]:
Y + Z

NameError: name 'Z' is not defined

In [15]:
Z.cuda(1) is Z

NameError: name 'Z' is not defined

神经网络与GPU

In [17]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu()) # 将模型放入到GPU上

net(X)

tensor([[0.0183],
        [0.0183]], grad_fn=<AddmmBackward>)

确认模型参数存储在同一个GPU上

In [18]:
net[0].weight.data.device

device(type='cpu')